In [1]:
from __future__ import print_function

import flask
import io
import keras
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.models import load_model
import numpy as np
from keras.utils.vis_utils import plot_model

# initialize our Flask application and the Keras model
app = flask.Flask(__name__)
model = None

print("* Loading Keras... please wait will Keras model loading is complete")

batch_size = 64  # Batch size for training.
epochs = 500  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'music/data copy 2.txt'

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])


input_token_index = dict(
        [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
        [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


# plot the model
#plot_model(model, to_file='model.png', show_shapes=True)

# summarize model
#plot_model(encoder, to_file='encoder_lstm.png', show_shapes=True)
#plot_model(decoder_lstm, to_file='decoder_lstm.png', show_shapes=True)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.0)

keras.callbacks.EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=2,
                              verbose=0, mode='auto')

# Save model
model.save('l2n-500.h5')

# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


Using TensorFlow backend.


* Loading Keras... please wait will Keras model loading is complete
Epoch 1/500
314/314 [==============================] - 4s 11ms/step - loss: 1.7512
Epoch 2/500
314/314 [==============================] - 3s 8ms/step - loss: 1.3030
Epoch 3/500
314/314 [==============================] - 3s 8ms/step - loss: 1.2810
Epoch 4/500
314/314 [==============================] - 3s 9ms/step - loss: 1.2694
Epoch 5/500
314/314 [==============================] - 3s 8ms/step - loss: 1.2628
Epoch 6/500
314/314 [==============================] - 3s 8ms/step - loss: 1.2603
Epoch 7/500
314/314 [==============================] - 3s 8ms/step - loss: 1.2493
Epoch 8/500
314/314 [==============================] - 3s 8ms/step - loss: 1.2413
Epoch 9/500
314/314 [==============================] - 3s 8ms/step - loss: 1.2303
Epoch 10/500
314/314 [==============================] - 3s 8ms/step - loss: 1.2157
Epoch 11/500
314/314 [==============================] - 3s 8ms/step - loss: 1.2108
Epoch 12/500
314/314 [=====

314/314 [==============================] - 3s 9ms/step - loss: 0.3852
Epoch 99/500
314/314 [==============================] - 3s 10ms/step - loss: 0.3742
Epoch 100/500
314/314 [==============================] - 3s 9ms/step - loss: 0.3761
Epoch 101/500
314/314 [==============================] - 3s 8ms/step - loss: 0.3562
Epoch 102/500
314/314 [==============================] - 3s 8ms/step - loss: 0.3593
Epoch 103/500
314/314 [==============================] - 3s 8ms/step - loss: 0.3529
Epoch 104/500
314/314 [==============================] - 3s 8ms/step - loss: 0.3586
Epoch 105/500
314/314 [==============================] - 3s 8ms/step - loss: 0.3349
Epoch 106/500
314/314 [==============================] - 3s 8ms/step - loss: 0.3376
Epoch 107/500
314/314 [==============================] - 3s 8ms/step - loss: 0.3363
Epoch 108/500
314/314 [==============================] - 3s 8ms/step - loss: 0.3318
Epoch 109/500
314/314 [==============================] - 3s 8ms/step - loss: 0.3321
Epoch 

314/314 [==============================] - 3s 9ms/step - loss: 0.1235
Epoch 196/500
314/314 [==============================] - 3s 10ms/step - loss: 0.1149
Epoch 197/500
314/314 [==============================] - 3s 10ms/step - loss: 0.1090
Epoch 198/500
314/314 [==============================] - 3s 11ms/step - loss: 0.1087
Epoch 199/500
314/314 [==============================] - 3s 10ms/step - loss: 0.1128
Epoch 200/500
314/314 [==============================] - 4s 11ms/step - loss: 0.1125
Epoch 201/500
314/314 [==============================] - 4s 12ms/step - loss: 0.1121
Epoch 202/500
314/314 [==============================] - 4s 12ms/step - loss: 0.1034
Epoch 203/500
314/314 [==============================] - 4s 13ms/step - loss: 0.1084
Epoch 204/500
314/314 [==============================] - 3s 10ms/step - loss: 0.1108
Epoch 205/500
314/314 [==============================] - 3s 10ms/step - loss: 0.1048
Epoch 206/500
314/314 [==============================] - 4s 11ms/step - loss: 0.

314/314 [==============================] - 3s 8ms/step - loss: 0.0674
Epoch 292/500
314/314 [==============================] - 3s 9ms/step - loss: 0.0643
Epoch 293/500
314/314 [==============================] - 3s 8ms/step - loss: 0.0651
Epoch 294/500
314/314 [==============================] - 3s 8ms/step - loss: 0.0708
Epoch 295/500
314/314 [==============================] - 3s 8ms/step - loss: 0.0670
Epoch 296/500
314/314 [==============================] - 3s 8ms/step - loss: 0.0687
Epoch 297/500
314/314 [==============================] - 3s 8ms/step - loss: 0.0655
Epoch 298/500
314/314 [==============================] - 3s 8ms/step - loss: 0.0698
Epoch 299/500
314/314 [==============================] - 3s 8ms/step - loss: 0.0631
Epoch 300/500
314/314 [==============================] - 3s 8ms/step - loss: 0.0641
Epoch 301/500
314/314 [==============================] - 3s 8ms/step - loss: 0.0649
Epoch 302/500
314/314 [==============================] - 3s 8ms/step - loss: 0.0902
Epoch 

314/314 [==============================] - 3s 9ms/step - loss: 0.0546
Epoch 389/500
314/314 [==============================] - 3s 10ms/step - loss: 0.0488
Epoch 390/500
314/314 [==============================] - 3s 10ms/step - loss: 0.0505
Epoch 391/500
314/314 [==============================] - 3s 9ms/step - loss: 0.0509
Epoch 392/500
314/314 [==============================] - 3s 9ms/step - loss: 0.0535
Epoch 393/500
314/314 [==============================] - 3s 9ms/step - loss: 0.0479
Epoch 394/500
314/314 [==============================] - 3s 9ms/step - loss: 0.0482
Epoch 395/500
314/314 [==============================] - 3s 10ms/step - loss: 0.0511
Epoch 396/500
314/314 [==============================] - 3s 9ms/step - loss: 0.0521
Epoch 397/500
314/314 [==============================] - 3s 9ms/step - loss: 0.0499
Epoch 398/500
314/314 [==============================] - 2s 8ms/step - loss: 0.0556
Epoch 399/500
314/314 [==============================] - 3s 9ms/step - loss: 0.0527
Epo

314/314 [==============================] - 2s 8ms/step - loss: 0.0416
Epoch 486/500
314/314 [==============================] - 2s 8ms/step - loss: 0.0378
Epoch 487/500
314/314 [==============================] - 2s 8ms/step - loss: 0.0392
Epoch 488/500
314/314 [==============================] - 2s 8ms/step - loss: 0.0433
Epoch 489/500
314/314 [==============================] - 2s 8ms/step - loss: 0.0371
Epoch 490/500
314/314 [==============================] - 2s 8ms/step - loss: 0.0368
Epoch 491/500
314/314 [==============================] - 2s 8ms/step - loss: 0.0395
Epoch 492/500
314/314 [==============================] - 2s 8ms/step - loss: 0.0456
Epoch 493/500
314/314 [==============================] - 2s 8ms/step - loss: 0.0415
Epoch 494/500
314/314 [==============================] - 3s 8ms/step - loss: 0.0390
Epoch 495/500
314/314 [==============================] - 2s 8ms/step - loss: 0.0470
Epoch 496/500
314/314 [==============================] - 3s 8ms/step - loss: 0.0366
Epoch 

/Volumes/Mac/anaconda2/envs/komposer/lib/python2.7/site-packages/keras/engine/network.py:888: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


RuntimeError: Can't decrement id ref count (unable to extend file properly, errno = 28, error message = 'No space left on device')

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
def generate(lyrics):
    
    abc = "";
    input_lyrics = []
    input_chars = set()
    lines = lyrics.split('\n')
    for line in lines[: len(lines) - 1]:
        input_lyrics.append(line)
    
    encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
    
    for seq_index in range(len(input_lyrics)):
        
        for i in range(len(input_lyrics)):
            for t, char in enumerate(input_lyrics[seq_index]):
                encoder_input_data[i, t, input_token_index[char]] = 1.
        
        input_seq = encoder_input_data[seq_index: seq_index + 1]
        decoded_sentence = decode_sequence(input_seq)
        abc += decoded_sentence
        abc += "w: "+input_lyrics[seq_index]+"\n"
    
    return abc;

In [2]:
@app.route("/predict", methods=["POST"])
def predict():
    # initialize the data dictionary that will be returned from the
    # view
    data = {"success": False}

    if flask.request.method == "POST":
        if flask.request.form.get("lyrics"):
        
            lyrics = flask.request.form.get("lyrics")

            data["komposition"] = generate(lyrics)

            # indicate that the request was a success
            data["success"] = True

    # return the data dictionary as a JSON response
    return flask.jsonify(data)

In [ ]:
# if this is the main thread of execution first load the model and
# then start the server
if __name__ == "__main__":
    print("* Keras initialization completed successfully...")
    print(("* Starting Flask server..."
        "please wait until server has fully started"))
    app.run(host="127.0.0.1", port=5000)

* Keras initialization completed successfully...
* Starting Flask server...please wait until server has fully started
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
[2018-09-16 22:10:23,658] ERROR in app: Exception on /predict [POST]
Traceback (most recent call last):
  File "/Volumes/Mac/anaconda2/envs/komposer/lib/python2.7/site-packages/flask/app.py", line 2292, in wsgi_app
    response = self.full_dispatch_request()
  File "/Volumes/Mac/anaconda2/envs/komposer/lib/python2.7/site-packages/flask/app.py", line 1815, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Volumes/Mac/anaconda2/envs/komposer/lib/python2.7/site-packages/flask/app.py", line 1718, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Volumes/Mac/anaconda2/envs/komposer/lib/python2.7/site-packages/flask/app.py", line 1813, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Volumes/Mac/anaconda2/envs/komposer/lib/python2.7/site-packages/flask/app.py", line 1799, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-i